In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

In [14]:
input_path = "C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/"

In [15]:
import os
def count_files_in_folders(directory):
  for root, dirs, files in os.walk(directory):
    print(f"Folder: {root}")
    file_count = len(files)
    print(f"Number of files: {file_count}")
    print("-" * 20) # Replace 'path_to_your_directory' with the directory path you want to inspectdirectory_path = 'path_to_your_directory'
count_files_in_folders(input_path)

Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/
Number of files: 2
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train
Number of files: 0
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train\Eyes Closed
Number of files: 2969
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train\Forward
Number of files: 3537
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train\Left Mirror
Number of files: 3251
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train\Radio
Number of files: 3537
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/lisat_gaze_data_mixed/train\Rearview
Number of files: 3433
--------------------
Folder: C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_dat

In [29]:
from torch.utils.data import random_split

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])
}
full_dataset = datasets.ImageFolder(input_path + 
                                    '/train', 
                                    data_transforms['train'])

# Calculate the sizes of the training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders for training and validation sets
dataloaders = {
    'train': torch.utils.data.DataLoader(train_dataset,
                                         batch_size=32,
                                         shuffle=True,
                                         num_workers=2),
    'validation': torch.utils.data.DataLoader(val_dataset,
                                              batch_size=32,
                                              shuffle=True,
                                              num_workers=2)
}
print(len(train_dataset))
print(len(val_dataset))

24004
6001


In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [31]:
model =torchvision.models.mobilenet_v2(pretrained=True).to(device)
model

C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [32]:
model2 =torchvision.models.mobilenet_v2(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model2.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=8, bias=True),
            nn.Softmax(dim=1))

model2

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.classifier.parameters(),lr=0.0001)

In [37]:
class EarlyStopper:
    def __init__(self, patience=10, min_delta=0.5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_accuracy = 0

    def early_stop(self, current_accuracy):
        if (current_accuracy > self.best_accuracy + self.min_delta):
            self.best_accuracy = current_accuracy
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [38]:
weights_path='MNV2_weights.h5'

In [41]:
early_stopper = EarlyStopper(10,0.01)
def train_model(model2, criterion, optimizer, num_epochs=5):
    training_accuracies = []
    validation_accuracies = []
    training_accuracies.append(0)
    validation_accuracies.append(0)
    #optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model2.train().to(device) #added to device
            else:
                model2.eval().to(device)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model2(inputs).to(device)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                epoch_loss = running_loss / len(train_dataset)
                epoch_acc = running_corrects.double() / len(train_dataset)
                training_accuracies.append(epoch_acc.item()*100)
            else:
                epoch_loss = running_loss / len(val_dataset)
                epoch_acc = running_corrects.double() / len(val_dataset)
                validation_accuracies.append(epoch_acc.item()*100)
                if early_stopper.early_stop(epoch_acc.item()*100):
                    print('triggered ES Tacc: {:.4f}, Vacc: {:.4f}'.format(training_accuracies[-1], 
                                                                           validation_accuracies[-1]))            
                    return model,training_accuracies,validation_accuracies
            
                if epoch_acc.item()*100 >= early_stopper.best_accuracy:
                    #early_stopper.early_stop(epoch_acc.item()*100)
                    #early_stopper.best_accuracy = epoch_acc.item()*100
                    print('best accuracy {:.4f}'.format(early_stopper.best_accuracy))
                    torch.save(model.state_dict(), weights_path)
                    print(f"Model saved as it achieved the best validation accuracy so far {early_stopper.best_accuracy}" )

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model2,training_accuracies,validation_accuracies 

In [42]:
model_trained = train_model(model2, criterion, optimizer, num_epochs=100) #1 epoch for testing gpu

Epoch 1/100
----------
train loss: 1.8370, acc: 0.5147
best accuracy 58.0070
Model saved as it achieved the best validation accuracy so far 58.006998833527746
validation loss: 1.7832, acc: 0.5801
Epoch 2/100
----------
train loss: 1.7856, acc: 0.5641
best accuracy 59.4734
Model saved as it achieved the best validation accuracy so far 59.473421096483925
validation loss: 1.7477, acc: 0.5947
Epoch 3/100
----------
train loss: 1.7556, acc: 0.5857
best accuracy 62.2063
Model saved as it achieved the best validation accuracy so far 62.206298950174975
validation loss: 1.7236, acc: 0.6221
Epoch 4/100
----------
train loss: 1.7355, acc: 0.5985
best accuracy 63.0728
Model saved as it achieved the best validation accuracy so far 63.07282119646725
validation loss: 1.7066, acc: 0.6307
Epoch 5/100
----------
train loss: 1.7221, acc: 0.6089
best accuracy 63.5227
Model saved as it achieved the best validation accuracy so far 63.522746208965174
validation loss: 1.6896, acc: 0.6352
Epoch 6/100
---------

train loss: 1.5676, acc: 0.7275
validation loss: 1.5300, acc: 0.7664
Epoch 58/100
----------
train loss: 1.5692, acc: 0.7250
best accuracy 77.5704
Model saved as it achieved the best validation accuracy so far 77.57040493251125
validation loss: 1.5229, acc: 0.7757
Epoch 59/100
----------
train loss: 1.5676, acc: 0.7264
validation loss: 1.5311, acc: 0.7674
Epoch 60/100
----------
train loss: 1.5678, acc: 0.7259
validation loss: 1.5246, acc: 0.7709
Epoch 61/100
----------
train loss: 1.5690, acc: 0.7245
validation loss: 1.5283, acc: 0.7650
Epoch 62/100
----------
train loss: 1.5670, acc: 0.7255
validation loss: 1.5246, acc: 0.7727
Epoch 63/100
----------
train loss: 1.5673, acc: 0.7257
best accuracy 77.9370
Model saved as it achieved the best validation accuracy so far 77.93701049825029
validation loss: 1.5215, acc: 0.7794
Epoch 64/100
----------
train loss: 1.5670, acc: 0.7266
validation loss: 1.5225, acc: 0.7769
Epoch 65/100
----------
train loss: 1.5675, acc: 0.7261
validation loss: 1

In [44]:
weights_path = 'C:/Users/AS-GP/Desktop/MobileNet/MNV2_weights.h5'

AttributeError: 'tuple' object has no attribute 'state_dict'

In [ ]:
#Later to restore:
model.load_state_dict(torch.load(weights_path))
model.eval()